In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
pd.set_option("display.max_columns",45)

**Key**

In [3]:
Patient="Casefortesting"

**Read CNV plot**

In [4]:
cnv=pd.read_excel(Patient+".seg_review.xlsx",index_col=None)
len(cnv)

1505

**Find unique samples**

In [5]:
samples=list(cnv.Sample.unique())
samples

['Patient_A1', 'Patient_A2', 'Patient_A3', 'Patient_A4', 'Patient_G1']

**Create list of  CNVs that correpsonds to unique samples for Zip command**

In [6]:
cnvs=['CNV_'+str(i+1) for i in range(len(samples))]
cnvs

['CNV_1', 'CNV_2', 'CNV_3', 'CNV_4', 'CNV_5']

**Read the mutation file**

In [7]:
mut=pd.read_excel(Patient+"_table.xlsx",index_col=None)
mut["key2"]=mut.Key.str.split(":").apply(lambda x :x[1])
mut["chromosome"]=mut.key2.str.split(".").apply(lambda x :x[0])
mut["position"]=mut.Key.str.split(".").apply(lambda x :x[1])
mut['position'] = mut['position'].astype(float)
mut["chromosome"]=mut.chromosome.str.replace("chr","")
len(mut)

36

**Add  copy number of each sample in last column in loop**

In [8]:
for s,c in zip(samples,cnvs):
    c= cnv[cnv.Sample==s]   # selects each sample
    c=c[['Chromosome', 'Start', 'End','abscopy']]
    c.columns=['chromosome', 'start', 'end', s.split('_')[1]] # strips the full sample name to A1 etc
    c["chromosome"]=c.chromosome.str.replace("chr","")
    c_mut=mut.merge(c,on="chromosome", how="outer")
    c_mut['position'] = c_mut['position'].astype(float)
    c_mut=c_mut[c_mut["position"].between(c_mut.pop("start"),c_mut.pop("end"))]
    c_mut_select=c_mut[["chromosome","position",s.split('_')[1]]]
    mut=mut.merge(c_mut_select,how="left") # fiel is renamed as mut so it goes in loop with previous addition

mut.to_excel(Patient+"_copyadded_output.xlsx",index=None)
len(mut)

36

**"################################"**

**Alternate Method**

**Without Loop**

In [9]:
Patient="Casefortesting"
cnv=pd.read_excel(Patient+".seg_review.xlsx",index_col=None)
len(cnv)

1505

In [10]:
samples=list(cnv.Sample.unique())
samples

['Patient_A1', 'Patient_A2', 'Patient_A3', 'Patient_A4', 'Patient_G1']

In [11]:
cnv=cnv[['Chromosome', 'Start', 'End','abscopy']]
cnv.columns=['chromosome', 'start', 'end', 'abscopy']
cnv["chromosome"]=cnv.chromosome.str.replace("chr","")
cnv.head()

,chromosome,start,end,abscopy
0,1,61724,16906668,1.702907
1,1,16906668,17016566,2.050534
2,1,17016566,25588235,1.706452
3,1,25588235,25646966,1.139183
4,1,25646966,30737864,1.694665


In [12]:
mut=pd.read_excel(Patient+"_table.xlsx",index_col=None)
mut["key2"]=mut.Key.str.split(":").apply(lambda x :x[1])
mut["chromosome"]=mut.key2.str.split(".").apply(lambda x :x[0])
mut["position"]=mut.Key.str.split(".").apply(lambda x :x[1])
mut['position'] = mut['position'].astype(float)
mut["chromosome"]=mut.chromosome.str.replace("chr","")
len(cnv),len(mut)

(1505, 36)

In [13]:
#Step1
cnv_mut=mut.merge(cnv,on="chromosome", how="outer")
cnv_mut['position'] = cnv_mut['position'].astype(float) #change ojbect to float/int
#Step2
cnv_mut=cnv_mut[cnv_mut["position"].between(cnv_mut.pop("start"),cnv_mut.pop("end"))]
## subset only columns needed
cnv_mut_select=cnv_mut[["chromosome","position","abscopy"]]
#Step3
final=mut.merge(cnv_mut_select,how="left")

In [14]:
final1=final[['Key','abscopy']]

In [15]:
from itertools import cycle
samples = cycle(['A1','A2','A3','A4','G1'])
final1['Sample'] = [next(samples) for s in range(len(final1))]
final2=final1.pivot(index="Key", columns="Sample", values="abscopy")
final3=mut.merge(final2,how="left", on="Key")
len(final3)

/var/folders/19/hnbyffk54hd9bzf5m3n39710562ngs/T/ipykernel_20568/1904508241.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final1['Sample'] = [next(samples) for s in range(len(final1))]


36

In [16]:
final3.head()

,Key,Chromosome,Position,Ref_allele,Alt_allele,Gene,AAChange,Class,key2,chromosome,position,A1,A2,A3,A4,G1
0,CELSR2:chr1.109817339.G.T,chr1,109817339,G,T,CELSR2,E34_UTR_3,UTR_3,chr1.109817339.G.T,1,109817339.0,1.667862,1.701727,1.988940,1.817556,2.001387
1,TCHHL1:chr1.152059809.C.A,chr1,152059809,C,A,TCHHL1,G117*,nonsense,chr1.152059809.C.A,1,152059809.0,2.380063,2.316586,2.030732,2.185555,2.012516
2,DUSP27:chr1.167096074.AGCCCG.-,chr1,167096074,AGCCCG,-,DUSP27,E569_G571>G,proteinDel,chr1.167096074.AGCCCG.-,1,167096074.0,2.419988,2.323017,2.023706,2.205338,2.002775
3,SLC9A11:chr1.173505089.C.T,chr1,173505089,C,T,SLC9A11,K552_E15splice,splice,chr1.173505089.C.T,1,173505089.0,2.419988,2.323017,2.023706,2.205338,2.002775
4,IVNS1ABP:chr1.185269450.G.C,chr1,185269450,G,C,IVNS1ABP,S423*,nonsense,chr1.185269450.G.C,1,185269450.0,2.419988,2.323017,2.023706,2.205338,2.002775


In [17]:
final3.to_excel("Alternate_copyadded_output.xlsx",index=None)